## INLÄMNING DEL 2

# Bearbeta data
- Läser in data från csv-fil
- Kontrollerar dataset
- Konverterar nödvändiga datatyper till rätt typ

In [ ]:
import sys
sys.executable

In [ ]:
import pandas as pd
import numpy as np
from data_cleaner import load_data, DataCleaner
from visualization import summary_table, plot_hist, plot_all_num_distributions, plot_group_box, plot_proportion_bar
import matplotlib.pyplot as plt


# 1. Ladda & rensa
df_raw = load_data("data/health_study_dataset.csv")
df = DataCleaner(df_raw).process()



### Datakontroll och förbehandling

En kvalitetskontroll genomfördes av datasetet.  
- Kolumnerna `sex` och `smoker` har kontrollerats och konverterats till kategoriska variabler.
- Kolumnen `disease` bestod endast av värdena 0 och 1 och konverterades därför till en boolesk variabel.  
- Kontroll av `id` visade att samtliga observationer är unika, vilket innebär att inga dubbletter förekommer.  

Datasetet är därmed välstrukturerat och lämpligt för vidare analys.

# Beskrivande analys

In [ ]:
from visualization import summary_table, plot_hist, plot_all_num_distributions, plot_group_box, plot_proportion_bar, plot_all_categorical_bars
import matplotlib.pyplot as plt

# 2. Sammanfattningstabell
summary = summary_table(df)
display(summary)

# 3. Alla numeriska distributioner
plot_all_num_distributions(df)

# 4. Alla kategoriska fördelningar
plot_all_categorical_bars(df)


## Tolkning av data

Den inledande analysen visar att datasetet överlag innehåller rimliga och realistiska värden, där variablerna uppvisar ungefär **normalfördelade** mönster. Det förekommer vissa extremvärden, särskilt i vikt, blodtryck och kolesterol, men dessa ligger inom tänkbara medicinska intervall och bedöms inte vara resultat av felregistrering. Därför behålls samtliga observationer inför vidare analys.

Utöver detta uppvisar de kategoriska variablerna en tydlig **obalans i gruppstorlekar**, särskilt för rökstatus och sjukdom. Andelen rökare är relativt liten och endast en mindre del av deltagarna är sjuka. Detta kan minska den statistiska styrkan vid gruppjämförelser, och resultaten bör därför tolkas med viss försiktighet, framför allt för den sjukdomsdrabbade gruppen.

Utöver detta visar de kategoriska variablerna en **ojämn gruppfördelning**, framför allt avseende rökstatus och sjukdomsförekomst. Andelen rökare är betydligt lägre än andelen icke-rökare, och endast en liten del av deltagarna är klassificerade som sjuka. Denna obalans innebär att vissa grupper har få observationer, vilket kan påverka den statistiska styrkan och göra jämförelser mellan grupper mer känsliga för slumpvariation. Vid senare analyser bör därför gruppstorlekar tas i beaktande, särskilt när man tolkar resultat för den sjukdomsdrabbade gruppen.

# Simulering kopplad till caset

In [ ]:
from confidence_intervals import ci_prop_normal

# andel i verklig data
real_prop = (df["disease"] == True).mean()
n_data = len(df)

# CI för verklig andel
level = 0.95
ci_low, ci_high, sd, z = ci_prop_normal(real_prop, n_data, level=level)

# Simulering
rng = np.random.default_rng(42)
n_sim=1000
simulated = rng.random(n_sim) < real_prop #Create a list with True and False values similar to real_prop
#simulated = np.random.choice([True, False], size=1000, p=[real_prop, 1-real_prop]) # same but slower
sim_prop = simulated.mean()

print(f"Andel i data: {real_prop*100:.2f}%")
print(f"Andel i simulering: {sim_prop*100:.2f}%")
print(f"{int(level*100)}% CI (normalapprox) för verklig andel: [{ci_low*100:.2f}%, {ci_high*100:.2f}%]")
#print(f"z = {z:.3f}, SD = {sd*100:.2f}%")
print("Ligger simulerad andel inom bandet?",
      "Ja" if ci_low <= sim_prop <= ci_high else "Nej")

### Tolkning av simulering:
 
Den simulerade andelen (6,30 %) ligger inom det 95-procentiga konfidensintervall som man kan förvänta sig kring den verkliga andelen (5,88 %). Eftersom simuleringen bygger på ett stort antal personer (1000 stycken) och sjukdomsförekomsten är låg, är det naturligt att andelen varierar något. Skillnaden är därför helt rimlig och beror på slumpen – simuleringen matchar datan väl.

In [ ]:
from health_analyzer import HealthAnalyzer

# 2. Skapa analyzer-objekt
analyzer = HealthAnalyzer(df, bp_col="systolic_bp")

fig, axes = analyzer.plot_bp_multi_boxgrid(
    group_cols=["sex", "smoker", "disease"]   # valfritt
)
plt.show()

# for col in ["sex", "smoker", "disease"]:
#     print(f"\nSummary for systolic_bp grouped by {col}:")
#     display(analyzer.bp_summary_by_group(col))


# Kön
summary_sex = analyzer.analyze_bp_by_group("sex", order=["F", "M"])

# Rökare
summary_smoker = analyzer.analyze_bp_by_group("smoker", order=["No", "Yes"])

# Sjukdom
summary_disease = analyzer.analyze_bp_by_group("disease", order=[False, True])





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import norm, ttest_ind, t as t_dist
import io
from contextlib import redirect_stdout
#import seaborn as sns

pd.options.display.float_format = "{:.2f}".format

# Välj Seaborn-tema
#sns.set_theme(style="whitegrid")


# Systoliskt blodtryck

Vidare analyser görs för Systoliskt blodtryck då det kan antas vara utfallsvariabel, alltså andra värden kan antas vara en förklarande variabel för det systoliksa blodtrycket.

## Konfidensintervall - Systolisk blodtryck

In [12]:
from confidence_intervals import ci_methods_table #ci_norm, ci_mean_boot_perc, ci_mean_boot_bca,

# CI för systoliskt blodtryck med tre metoder

ci_df = ci_methods_table(df["systolic_bp"], confidence=0.95, n_boot=10_000, random_state=42)
display(ci_df.style.format({"Medel": "{:.2f}", "KI nedre": "{:.2f}", "KI övre": "{:.2f}"}))




,Metod,Medel,KI nedre,KI övre,Nivå (%)
0,Normalapproximation,149.18,148.29,150.07,95
1,Bootstrap (percentil),149.18,148.31,150.05,95
2,Bootstrap (BCa),149.18,148.30,150.03,95


### Tolkning av skattning av konfidensintervall för Systolisk blodtryck
Resultaten från de tre metoderna visar liknande 95 %-konfidensintervall för medelvärdet av systoliskt blodtryck, vilket tyder på en stabil skattning som inte är känslig för metodval.

Normalapproximationen fungerar väl då urvalet är stort och blodtrycket är ungefär normalfördelat (se figur "Distribution of systolic blood pressure" ).

Bootstrapmetoderna ger en icke-parametrisk kontroll, där BCa justerar för eventuell skevhet. De överlappande intervallen antyder att skillnaderna främst beror på slumpvariation.

## Test hur bra CI-metoderna för konfidensintervall är

In [ ]:
from ci_simulation import evaluate_ci_methods

# res = evaluate_ci_methods(df["systolic_bp"],
#                           n=50, trials=100, confidence=0.95)

# display(res)


# # Test the CI-methods
# def covers_true_mean(series, method="normal", n=40, trials=200, confidence=0.95, n_boot=20_000, random_state=42):
#     """
#     Calculates the coverage for a given CI method.

#     Args:
#         series: Data to analyze (pd.Series or array)
#         method: Which CI method to use ('normal', 'bootstrap', or 'bca')
#         n: Sample size for each simulation
#         trials: Number of simulations to run
#         confidence: Confidence level (e.g., 0.95 for 95% CI)
#         random_state: Random seed for reproducibility

#     Returns:
#         Dict with the method's performance:
#         - coverage: Proportion of intervals that contain the population mean
#         - avg_width: Average interval width
#         - std_width: Standard deviation of the interval widths
#     """
#     x = np.asarray(series, dtype=float)
#     x = x[~np.isnan(x)]
#     true_mean = float(x.mean())
#     rng = np.random.default_rng(random_state)

#     hits = 0 # Counter for number of hits (intervals that cover true_mean)
#     widths = [] # Collecting interval widths

#     # Select CI method
#     func_map = {
#         "normal": ci_norm,
#         "bootstrap": ci_mean_boot_perc,
#         "bca": ci_mean_boot_bca
#     }
#     ci_func = func_map.get(method)
#     if ci_func is None:
#         raise ValueError(f"Unknown method: {method}")
    
#     # Run simulations
#     for _ in range(trials):
#         sample = rng.choice(x, size=n, replace=True)
        
#         # Calculate CI 
#         if method in ["bootstrap", "bca"]:
#             out = ci_func(sample, confidence, n_boot=n_boot) #, random_state=random_state så inte exakt samma slumpsekvens används för bootstrap-resamplingen.
#         else:
#             out = ci_func(sample, confidence)

#         # Unpack the result
#         _, (lo, hi), _ = out
        
#         #Update statistics
#         hits += (lo <= true_mean <= hi)
#         widths.append(hi - lo)
    
#     return {
#         'method': method,
#         'coverage': hits / trials,
#         'avg_width': float(np.mean(widths)),
#         'std_width': float(np.std(widths))
#     }

# def compare_ci_methods(data, methods=('normal','bootstrap','bca'), 
#                        n=40, trials=200, confidence=0.95,
#                        n_boot=20_000, random_state=42):
#     """
#     Compare multiple CI methods through simulation.
    
#     Args:
#         data: Dataset to analyze
#         methods: List of methods to compare
#         n, trials, confidence: Passed on to covers_true_mean
    
#     Returns:
#         DataFrame with results for all methods
#     """
#     results = [covers_true_mean(data, 
#                                 method=m, 
#                                 n=n, 
#                                 trials=trials, 
#                                 confidence=confidence,
#                                 n_boot=n_boot,
#                                 random_state=random_state
#                                 ) for m in methods]
#     result_df = pd.DataFrame(results).set_index('method')
#     return result_df


# res = compare_ci_methods(df['systolic_bp'],
#                          n=50, 
#                          trials=100, 
#                          confidence=0.95,
#                          n_boot=10_000,
#                          random_state=42)

# res_sv = (
#     res.reset_index()  # Reset index to a column
#        .rename(columns={
#            "method": "Metod",
#            "coverage": "Täckningsgrad",
#            "avg_width": "Genomsnittlig bredd",
#            "std_width": "SD för bredd"
#        })
#        .round(3)
# )
# display(res_sv.style.format({
#     "Täckningsgrad": "{:.2f}",
#     "Genomsnittlig bredd": "{:.3f}",
#     "SD för bredd": "{:.3f}"
# }).set_caption("Jämförelse av konfidensintervall-metoder för Systolisk blodtryck"))



### Tolkning av test av olika metoder för skattning av konfidensintervall

Vid 95 % konfidensnivå täcker alla metoder det sanna medelvärdet ungefär så ofta som förväntat (≈ 0,95), vilket visar att både normalapproximationen och bootstrapmetoderna fungerar väl för systoliskt blodtryck i detta urval. Skillnaderna i täckningsgrad och intervallbredd är små, vilket tyder på att metoderna ger likvärdiga resultat.

BCa-metoden är teoretiskt mest avancerad eftersom den korrigerar för bias och skevhet, men här ger den ingen tydlig förbättring eftersom datan är symmetrisk och urvalet tillräckligt stort.

## Hypotesprövning

Hypotes: Rökare har högre medel-blodtryck än icke-rökare

H0​: Medelvärdet av systoliskt blodtryck är detsamma för rökare och icke-rökare. dvs medel(Yes)​−medel(No)​=0 (ingen skillnad) 

In [ ]:
# Function definitions for testing mean differences: bootstrap and Welch’s t-test

def bootstrap_mean_diff(group1, group2, confidence=0.95, n_boot=10_000, 
                        two_sided=False, random_state=42):
    """
    Bootstrap test for the difference in means between two groups.
    Returns the observed difference, p-value, confidence interval, and bootstrap distribution.
    """

    # Convert to clean NumPy arrays (remove NaNs)
    x1 = np.asarray(group1, dtype=float)
    x2 = np.asarray(group2, dtype=float)
    x1 = x1[~np.isnan(x1)]
    x2 = x2[~np.isnan(x2)]

    obs_diff = x1.mean() - x2.mean()

    rng = np.random.default_rng(random_state)
    boot_diffs = np.empty(n_boot)

    # Bootstrap resampling
    for i in range(n_boot):
        s1 = rng.choice(x1, size=len(x1), replace=True)
        s2 = rng.choice(x2, size=len(x2), replace=True)
        boot_diffs[i] = s1.mean() - s2.mean()

    # p-value
    if two_sided:
        p_boot = np.mean(np.abs(boot_diffs) >= np.abs(obs_diff))
    else:
        p_boot = np.mean(boot_diffs >= obs_diff)

    # Confidence interval
    alpha = 1 - confidence
    ci_low, ci_high = np.percentile(boot_diffs, [100*alpha/2, 100*(1-alpha/2)])

    return obs_diff, p_boot, (ci_low, ci_high), boot_diffs


#t-test for difference in means between two groups
def welch_t_test_mean_diff(group1, group2, confidence = 0.95, two_sided=False):
    """
    Welch's t-test for the difference in means between two groups.
    Returns:
        diff (difference in means),
        p_val (p-value),
        (ci_low, ci_high) (confidence interval),
        t_stat (t-statistic),
        dof (degrees of freedom).
    """
    # Convert to clean NumPy arrays (remove NaNs)
    x1 = np.asarray(group1, dtype=float)
    x2 = np.asarray(group2, dtype=float)
    x1 = x1[~np.isnan(x1)]
    x2 = x2[~np.isnan(x2)]

    # Means & variances
    m1, m2 = x1.mean(), x2.mean()
    v1, v2 = x1.var(ddof=1), x2.var(ddof=1)
    n1, n2 = len(x1), len(x2)

    # Observed difference in means
    diff = m1 - m2

    # Welch's t-test using SciPy
    t_stat, p_two_sided = ttest_ind(x1, x2, equal_var=False)

    #Handle one-sided vs two-sided p-value
    if two_sided:
        p_value = p_two_sided
    else:
        # One-sided test: tests the hypothesis diff > 0
        p_value = p_two_sided/2 if t_stat > 0 else 1 - p_two_sided/2
 
    # Degrees of freedom (Welch–Satterthwaite)
    dof = (v1/n1 + v2/n2)**2 / ((v1**2)/((n1**2)*(n1-1)) + (v2**2)/((n2**2)*(n2-1)))

    # Standard error and critical t-value
    se = np.sqrt(v1/n1 + v2/n2)
    alpha = 1 - confidence
    t_crit = t_dist.ppf(1 - alpha/2, dof) if two_sided else t_dist.ppf(1 - alpha, dof)

    # Confidence interval
    ci_low = diff - t_crit * se
    ci_high = diff + t_crit * se

    return diff, p_value, (ci_low, ci_high), t_stat, dof, se



## Tillämpning och visualisering av hypotesprövning

In [ ]:
# Application and visualization of bootstrap and Welch’s t-test for mean difference

#Bootstrap 
def plot_bootstrap_dist(boot_diffs, obs_diff, ci,  p_boot=None, xlabel="Skillnad (grupp1 − grupp2)"):
    ci_low, ci_high = ci
    
    fig, ax = plt.subplots(figsize=(7,4))
    ax.hist(boot_diffs, bins=50, density=True, alpha=0.6, label="Bootstrapfördelning")
    
    # Observed difference
    ax.axvline(obs_diff, linewidth=2, label=f'Observerad diff = {obs_diff:.3f}')

    # Confidence interval
    ax.axvline(ci_low, linestyle='--', label=f"KI: ({ci_low:.3f}, {ci_high:.3f})")
    ax.axvline(ci_high, linestyle='--')
    
    # Zero difference (for intuition)
    ax.axvline(0, linestyle=':', label='0')

    ax.set_title("Bootstrapfördelning av skillnad i medelvärde")
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Täthet")
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()


def plot_t_distribution(t_stat, dof, confidence=0.95, two_sided=False):
    """
    Plottar t-fördelningen med markerat kritiskt område och observerat t-värde.
    two_sided=False = ensidigt test (H1: diff > 0).
    """

    alpha = 1 - confidence
    
    # Determine critical t depending on one-sided/two-sided test
    if two_sided:
        t_crit = t_dist.ppf(1 - alpha/2, dof)
        crit_left = -t_crit
        crit_right = t_crit
    else:
        t_crit = t_dist.ppf(1 - alpha, dof)
        crit_left = None
        crit_right = t_crit

    # x-axis
    xs = np.linspace(-5, 5, 1000)
    ys = t_dist.pdf(xs, dof)

    fig, ax = plt.subplots(figsize=(7,4))
    ax.plot(xs, ys, label=f"t-fördelning (df = {dof:.1f})")

    # Shade the critical region
    if two_sided:
        ax.fill_between(xs, 0, ys, where=(xs <= crit_left), alpha=0.2, label="Kritiskt område (α/2)")
        ax.fill_between(xs, 0, ys, where=(xs >= crit_right), alpha=0.2)
    else:
        ax.fill_between(xs, 0, ys, where=(xs >= crit_right), alpha=0.2, label="Kritiskt område (α)")
    
    # Mark the critical t-value
    if two_sided:
        ax.axvline(crit_left, linestyle='--', label=f"−t_crit = {crit_left:.3f}")
    ax.axvline(crit_right, linestyle='--', label=f"t_crit = {t_crit:.3f}")

    # Mark the observed t-value
    ax.axvline(t_stat, linewidth=2, label=f"t_obs = {t_stat:.3f}", color="black")

    # Graph settings
    title = "t-fördelning med kritiskt område (ensidigt test)" if not two_sided else "t-fördelning med kritiskt område (tvåsidigt test)"
    ax.set_title(title)
    ax.set_xlabel("t-värde")
    ax.set_ylabel("Täthet")
    ax.grid(alpha=0.3)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()


def plot_mean_diff_normal(diff, se, ci, xlabel="Skillnad (grupp1 − grupp2)"):
    """
    Visualizes the normal approximation of the difference in means.
    'diff' = observed difference.
    'se'   = standard error of the difference.
    'ci'   = (ci_low, ci_high) from your test.
    """
    
    ci_low, ci_high = ci

    # Create x-axis around diff
    xs = np.linspace(diff - 5*se, diff + 5*se, 400)
    ys = (1/(np.sqrt(2*np.pi)*se)) * np.exp(-(xs - diff)**2 / (2*se**2))

    plt.figure(figsize=(7,4))
    plt.plot(xs, ys, label="Normalapproximation")
    
    # Observed difference
    plt.axvline(diff, linewidth=2, label=f"Observerad diff = {diff:.3f}")

    # Confidence interval
    plt.axvline(ci_low, linestyle='--', label=f"KI: ({ci_low:.3f}, {ci_high:.3f})")
    plt.axvline(ci_high, linestyle='--')
    plt.fill_between(xs, 0, ys, where=(xs>=ci_low)&(xs<=ci_high), alpha=0.2)

    # Zero difference
    plt.axvline(0, linestyle=':', label="0")

    plt.title("Normalapproximerad fördelning av skillnad i medelvärde")
    plt.xlabel(xlabel)
    plt.ylabel("Täthet")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Plot the functions for Systolic bp
smoker_yes = df.loc[df["smoker"] == "Yes", "systolic_bp"]
smoker_no = df.loc[df["smoker"] == "No", "systolic_bp"]

obs_diff, p_boot, (ci_low_b, ci_high_b), boot_diffs = bootstrap_mean_diff(
    smoker_yes, smoker_no,
    confidence=0.95,
    n_boot=10_000,
    two_sided=False  # Tests only whether smokers have higher blood pressure
)

diff, p_value, (ci_low_wt, ci_high_wt), t_stat, dof, se = welch_t_test_mean_diff(
    smoker_yes, smoker_no,
    confidence=0.95,
    two_sided=False  # Expecting smokers > non-smokers
)

# Display of results table
res_tests = pd.DataFrame({
    "Metod": ["Bootstrap", "Welch t-test"],
    "Observerad skillnad": [obs_diff, diff],
    "p-värde": [p_boot, p_value],
    "KI nedre": [ci_low_b, ci_low_wt],
    "KI övre": [ci_high_b, ci_high_wt]
}).round(3)

display(
    res_tests.style
    .set_caption("Jämförelse mellan bootstrap och Welch’s t-test")
    .format({
        "Observerad skillnad": "{:.3f}",
        "p-värde": "{:.3f}",
        "KI nedre": "{:.3f}",
        "KI övre": "{:.3f}"
    })
    .set_table_styles([
        {"selector": "caption",
         "props": [("caption-side", "top"), ("font-weight", "bold"), ("font-size", "110%")]},
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "td", "props": [("text-align", "center")]}
    ])
)

ffig, axes = plt.subplots(1, 3, figsize=(15, 4))

# --- 1️⃣ t-fördelning ---
alpha = 0.05
t_crit = t_dist.ppf(1 - alpha, dof)
xs = np.linspace(-5, 5, 400)
ys = t_dist.pdf(xs, dof)
axes[0].plot(xs, ys, label=f"t-fördelning (df = {dof:.1f})")
axes[0].fill_between(xs, 0, ys, where=(xs >= t_crit), alpha=0.2, label="Kritiskt område (α)")
axes[0].axvline(t_crit, ls='--', label=f"t_crit = {t_crit:.3f}")
axes[0].axvline(t_stat, lw=2, label=f"t_obs = {t_stat:.3f}") #color="black"
axes[0].axvline(0, ls=':', label="0")
axes[0].set_title("t-fördelning med kritiskt område")
axes[0].set_xlabel("Skillnad i systoliskt blodtryck (mmHg)")
axes[0].set_ylabel("Täthet")
axes[0].grid(alpha=0.3)
axes[0].legend(loc="upper left", fontsize=8)

# --- 2️⃣ Normalapproximation ---
xs = np.linspace(diff - 5*se, diff + 5*se, 400)
ys = (1/(np.sqrt(2*np.pi)*se)) * np.exp(-(xs - diff)**2 / (2*se**2))
axes[1].plot(xs, ys, label="Normalapproximation")
axes[1].axvline(diff, lw=2, label=f"Observerad diff = {diff:.3f}")
axes[1].axvline(ci_low_wt, ls='--')
axes[1].axvline(ci_high_wt, ls='--', label=f"KI: [{ci_low_wt:.3f}, {ci_high_wt:.3f}]")
axes[1].axvline(0, ls=':', label='0')
axes[1].fill_between(xs, 0, ys, where=(xs >= ci_low_wt) & (xs <= ci_high_wt), alpha=0.2)
axes[1].set_title("Normalapproximerad fördelning av skillnad i medelvärde")
axes[1].set_xlabel("Skillnad i systoliskt blodtryck (mmHg)")
axes[1].set_ylabel("Täthet")
axes[1].grid(alpha=0.3)
axes[1].legend(loc="upper left", fontsize=8)

# --- 3️⃣ Bootstrapfördelning ---
axes[2].hist(boot_diffs, bins=50, density=True, alpha=0.6, label="Bootstrapfördelning")
axes[2].axvline(obs_diff, lw=2, label=f"Observerad diff = {obs_diff:.3f}")
axes[2].axvline(ci_low_b, ls='--')
axes[2].axvline(ci_high_b, ls='--', label=f"KI: [{ci_low_b:.3f}, {ci_high_b:.3f}]")
axes[2].axvline(0, ls=':', label="0")
axes[2].set_title("Bootstrapfördelning av skillnad i medelvärde")
axes[2].set_xlabel("Skillnad i systoliskt blodtryck (mmHg)")
axes[2].set_ylabel("Täthet")
axes[2].grid(alpha=0.3)
axes[2].legend(loc="upper left", fontsize=8)

plt.tight_layout()
plt.show()

# plot_t_distribution(t_stat, dof, confidence = 0.95, two_sided=False )
# plot_mean_diff_normal(diff, se, (ci_low_wt, ci_high_wt), xlabel="Skillnad i systoliskt blodtryck (mmHg)")
# plot_bootstrap_dist(boot_diffs, obs_diff, (ci_low_b, ci_high_b), p_boot=p_boot, xlabel="Skillnad i systoliskt blodtryck (mmHg)")



### Tolkning

Både bootstrapmetoden och Welch’s t-test ger mycket liknande resultat för skillnaden i medelvärde av systoliskt blodtryck mellan rökare och icke-rökare.
Den observerade skillnaden är liten (≈ 0,47 mmHg), och båda metoderna ger konfidensintervall som inkluderar noll. Detta innebär att ingen statistiskt säkerställd skillnad påvisas vid 95 % konfidensnivå.

De tre graferna visar fördelningar som är centrerade nära noll, vilket stämmer överens med att den observerade skillnaden är liten. Symmetrin i t- och normalfördelningen följer av antagandena i metoderna, medan bootstrapfördelningen visar att resultaten är stabila även utan sådana antaganden.

P-värdena (0,50 för bootstrap och 0,33 för Welch’s t-test) ligger långt över den vedertagna signifikansgränsen på 0,05. Nollhypotesen kan därför inte förkastas, vilket innebär att resultaten inte tyder på någon verklig skillnad i systoliskt blodtryck mellan rökare och icke-rökare.

Sammantaget visar både bootstrapmetoden och Welch’s t-test att eventuella skillnader i systoliskt blodtryck mellan rökare och icke-rökare sannolikt beror på slumpmässig variation snarare än en verklig effekt.

## Kontrollera hur säkert hypotestestet är, hittar en skillnad när den finns (power)

In [ ]:
# ==============================================
# Simulated Power Function
# ==============================================
def simulate_power_welch(
    group1, group2,
    effect=1.0,           # true difference in means: group1 - group2 (mmHg)
    n1=None, n2=None,     # sample sizes; None = keep current sizes
    trials=1000,
    alpha=0.05,
    one_sided=True,
    random_state=42
    ):
    """
    Power = the proportion of simulations where p < alpha given the true effect ‘effect’.
    Preserves shape/variance by resampling the centered data and shifting the mean in group1 by ‘effect’.
    Test: Welch’s t-test (equal_var=False).
    """
    rng = np.random.default_rng(random_state)

    # Convert to clean NumPy arrays (remove NaNs)
    x1 = np.asarray(group1, dtype=float)
    x2 = np.asarray(group2, dtype=float)
    x1 = x1[~np.isnan(x1)]
    x2 = x2[~np.isnan(x2)]

    # center to preserve shape/variance
    x1c = x1 - x1.mean()
    x2c = x2 - x2.mean()

    if n1 is None: n1 = len(x1) # Allows other sizes, default = current lengths
    if n2 is None: n2 = len(x2)

    hits = 0
    for _ in range(trials):
        # resample and shift group1 by ‘effect’
        s1 = rng.choice(x1c, size=n1, replace=True) + x1.mean() + effect
        s2 = rng.choice(x2c, size=n2, replace=True) + x2.mean()

        t_stat, p_two = ttest_ind(s1, s2, equal_var=False)

        if one_sided:
            # H1: mean(group1) > mean(group2)  (adjust the sign if needed)
            p = p_two/2 if t_stat > 0 else 1 - p_two/2
        else:
            p = p_two

        hits += (p < alpha)

    return hits / trials

# =====================================================================
# Functions for calculating what is required to achieve Power = 80% 
# Analytical calculation, not simulation, assuming normal approximation
# =====================================================================

# minimum Δ for a given N (and power)
def required_effect_for_power(group1, group2, 
                              total_n=None, power=0.8,
                              alpha=0.05, one_sided=True):
    """
    Beräknar minsta effektstorlek (Δ_min i mmHg) som krävs för att nå 'power'
    med ett totalt sample N (eller nuvarande N om total_n=None).
    Tar hänsyn till proportionen mellan grupperna.
    """
    x1 = np.asarray(group1, float); x1 = x1[~np.isnan(x1)]
    x2 = np.asarray(group2, float); x2 = x2[~np.isnan(x2)]
    
    # Pooled SD (shared sigma)
    all_x = np.concatenate([x1, x2])
    sigma = all_x.std(ddof=1)

    # Group proportions (use the current distribution)
    n1 = len(x1); n2 = len(x2)
    if total_n is None:
        total_n = n1 + n2
    p = n1 / (n1 + n2)   # proportion of smokers (group 1)
    
    C = 1/p + 1/(1-p)    # factor from 1/(pN) + 1/((1–p)N)

    # Critical z-values
    if one_sided:
        z_crit = norm.ppf(1 - alpha)
    else:
        z_crit = norm.ppf(1 - alpha/2)
    z_power = norm.ppf(power)  # e.g., ≈ 0.84 at power 0.80

    # Minimum Δ:
    delta_min = sigma * np.sqrt(C / total_n) * (z_crit + z_power)
    return float(delta_min)

def required_n_for_power(group1, group2,
                         delta, power=0.8,
                         alpha=0.05, one_sided=True):
    """
    Beräknar total N (och fördelningen per grupp) som krävs för att nå 'power'
    givet en sann effekt 'delta' (t.ex. obs_diff).
    """
    x1 = np.asarray(group1, float); x1 = x1[~np.isnan(x1)]
    x2 = np.asarray(group2, float); x2 = x2[~np.isnan(x2)]
    
    # Pooled standard deviation
    all_x = np.concatenate([x1, x2])
    sigma = all_x.std(ddof=1)
    
    # Proportion between the groups (keep the same as in the data)
    n1_now = len(x1); n2_now = len(x2)
    p = n1_now / (n1_now + n2_now)   # proportion of smokers
    C = 1/p + 1/(1-p)
    
    # z-value
    if one_sided:
        z_crit = norm.ppf(1 - alpha)
    else:
        z_crit = norm.ppf(1 - alpha/2)
    z_power = norm.ppf(power)

    # Solve for N from the formula:
    N_total = (sigma**2 * C * (z_crit + z_power)**2) / (delta**2)
    
    # Split N_total into groups according to the same proportion
    n1 = int(round(p * N_total))
    n2 = int(round(N_total - n1))
    
    return int(round(N_total)), n1, n2


# ===========================================================
# Power for the actual sample
# ===========================================================

# smoker_yes = df.loc[df["smoker"] == "Yes", "systolic_bp"]
# smoker_no  = df.loc[df["smoker"] == "No",  "systolic_bp"]
# obs_diff = smoker_yes.mean() - smoker_no.mean() #done earlier using float conversion and removal of NaNs

effect_chosen = obs_diff              # uses the observed difference as the “true” effect
real_total_n = len(smoker_yes) + len(smoker_no)
real_power = simulate_power_welch(
    smoker_yes, smoker_no,
    effect=effect_chosen,
    n1=len(smoker_yes),
    n2=len(smoker_no),
    trials=800, alpha=0.05, one_sided=True
)
print(f"Power för verkliga stickprovet (N = {real_total_n}, observerad skillnad {obs_diff:.3f}): {real_power:.3f}")

# ============================================================
# Power as a function of effect size (Δ) with the current n
# ============================================================

effects = np.arange(0.0, 3.1, 0.5)  # 0–3 mmHg i steg om 0.5
power_welch = [
    simulate_power_welch(smoker_yes, smoker_no, effect=d,
                         trials=800, alpha=0.05, one_sided=True)
    for d in effects
]

target_power = 0.80
power_arr = np.array(power_welch)

plt.figure(figsize=(6,4))
plt.plot(effects, power_welch, marker="o", label="Simulerad power (Welch)")
plt.axhline(target_power, linestyle="--", label="80 % power")


# Mark the observed difference, i.e., the actual difference
power_obs = np.interp(obs_diff, effects, power_welch)

plt.scatter(obs_diff, power_obs, color="red", s=70, zorder=5)
plt.scatter([], [], color="red", s=70, zorder=5, label=f"Observerad power = {real_power:.2f}")
plt.title(f"Power vs effektstorlek (N = {real_total_n}, α = 0.05)")
plt.xlabel("Effektskillnad Δ (mmHg) för Rökare − Icke rökare")
plt.ylabel("Power")
plt.grid(True, alpha=0.3)
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

if power_arr.max() >= target_power:
    delta_at_80 = float(np.interp(target_power, power_arr, effects))
    print(
        f"För att nå {target_power*100:.0f} % power med nuvarande stickprovsstorlek\n"
        f"skulle den sanna effekten behöva vara ungefär Δ ≈ {delta_at_80:.2f} mmHg."
    )
else:
    print(
        f"Även vid den största testade effekten (Δ = {effects.max():.2f} mmHg)\n"
        f"når power inte upp till {target_power*100:.0f} %."
    )


# ===================================================================
# Power as a function of total sample size with unbalanced groups
# ===================================================================
prop_of_total= len(smoker_yes)/(len(smoker_yes)+len(smoker_no)) #proportion of the total

target_power = 0.80

# Test different sample sizes (smokers + non-smokers)
totals_n = np.arange(200, 1601, 200)      # 20, 40, ..., 200 per grupp
power_tot = []
n1_list = []
n2_list = []

for N in totals_n:
    # Split N i n1 (smokers) och n2 (non-smokers) using the same proportion as in the data
    n1 = int(round(N * prop_of_total))
    n2 = N - n1
    n1 = max(n1, 5); n2 = max(n2, 5) # If the number is less than 5, force it to 5 (treated as the minimum)

    power_n = simulate_power_welch(
        smoker_yes, smoker_no,
        effect=effect_chosen,
        n1=n1, n2=n2,
        trials=600, alpha=0.05, one_sided=True
    )
    power_tot.append(power_n)
    n1_list.append(n1)
    n2_list.append(n2)
    
power_values = np.array(power_tot)
sample_sizes = np.array(totals_n)

# Plot power vs. total N (line and red dot for the actual N) 
plt.figure(figsize=(6,4))
plt.plot(sample_sizes, power_values, "o-", label="Simulerad power (Welch)")
plt.axhline(target_power, linestyle="--", label="80 % power")
plt.scatter(real_total_n, real_power, color="red", s=70, zorder=5)
plt.scatter([], [], color="red", s=70, label=f"Observerad power = {real_power:.2f}")
plt.title(f"Power vs total stickprovsstorlek (Δ = {effect_chosen:.2f} mmHg, α = 0.05)")
plt.xlabel("Totalt antal observationer (rökare + icke-rökare)")
plt.ylabel("Power")
plt.ylim(0, 1.05)
plt.grid(True, alpha=0.3)
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

if power_values.max() >= target_power:
    n_total_at_80 = float(np.interp(target_power, power_values, sample_sizes))
    n1_80 = int(round(n_total_at_80 * prop_of_total))
    n2_80 = int(round(n_total_at_80 - n1_80))
    print(
        f"För att nå {target_power*100:.0f} % power vid Δ = {effect_chosen:.2f} mmHg\n"
        f"skulle vi behöva ungefär N ≈ {int(round(n_total_at_80))} totalt "
        f"(≈ {n1_80} rökare och ≈ {n2_80} icke-rökare)."
    )
else:
    print(
        f"Power når inte upp till {target_power*100:.0f} % inom intervallet "
        f"N = {sample_sizes.min()}–{sample_sizes.max()}."
    )


# ===================================================================
# Calculations to achieve 80% power
# ===================================================================


delta_min = required_effect_for_power(smoker_yes, smoker_no,
                                      total_n=None,  # ditt nuvarande N
                                      power=0.80, alpha=0.05,
                                      one_sided=True)

prop_delta = delta_min/obs_diff
print("\nBeräknade power-värden för att nå 80% power:")
print(f"Med nuvarande stickprovsstorlek krävs en effekt på ca: \n Δ ≈ {delta_min:.2f} mmHg, ca {prop_delta:.2f} gånger större." )
#print(f"Observerad skillnad var Δ_obs ≈ {obs_diff:.2f} mmHg\n")


N_req, n1_req, n2_req = required_n_for_power(smoker_yes, smoker_no,
                                             delta=obs_diff,
                                             power=0.80, alpha=0.05,
                                             one_sided=True)

prop_N = N_req/real_total_n
print(f"Med nuvarande effektskillnad på Δ = {obs_diff:.2f} mmHg, skulle totalt: \n N ≈ {N_req:,} (≈ {n1_req} rökare och ≈ {n2_req} icke-rökare), ca {prop_N:.1f} gånger fler.")
#print(f"För att nå 80 % power vid Δ = {obs_diff:.2f} mmHg")
#print(f"skulle vi behöva totalt N ≈ {N_req} (≈ {n1_req} rökare och ≈ {n2_req} icke-rökare).")


### Tolkning
Den observerade skillnaden i systoliskt blodtryck mellan rökare och icke-rökare var mycket liten (≈ 0,47 mmHg) och poweranalysen visar att den här studien har låg power (≈ 20 %). Det betyder att sannolikheten att upptäcka en så liten effekt med N = 800 är låg.

Både simuleringarna och de analytiska powerberäkningarna visar att:
-   en betydligt större sann effekt (≈ 2–2,5 mmHg) hade krävts för att nå 80 % power med nuvarande stickprovsstorlek, och
-   om den verkliga skillnaden faktiskt är 0,47 mmHg skulle ett mycket stort stickprov (≈ 23 000 individer totalt) behövas för att nå 80 % power.

Detta innebär att studien är underpowered, vilket gör att den icke-signifikanta skillnaden inte kan tolkas som stöd för att ingen verklig effekt finns. Resultaten bör därför tolkas försiktigt, och studien kan inte ge starka slutsatser om skillnader i blodtryck mellan rökare och icke-rökare.


## METODVAL
### Deskriptiv statistik och visualiseringar

Histogram, boxplots och sammanfattande mått (medelvärde, median, standardavvikelse) anbändes för att beskriva variablernas fördelning. Detta är standardmetoder för att identifiera centraltendens, spridning och möjliga extremvärden.

Källor: Pandas descriptive statistics och Matplotlib histogram/boxplot.


### Simulering kopplad till caset

 Ett 95 % konfidensintervall beräknades för andelen personer med sjukdom med normalapproximation, vilket är en vanlig metod när stickprovet är stort. För att kontrollera rimligheten i resultatet simulerade jag ett nytt stickprov med samma sannolika andel och jämförde den simulerade andelen med intervallet. Syftet var att se om variationen kan förklaras av slumpen.

Källor: NumPy-dokumentation; Agresti (2018), Statistical Methods for the Social Sciences.

### Konfidensintervall (normalapproximation, bootstrap och BCa)

För att skatta konfidensintervallet för medelvärdet av systoliskt blodtryck användes tre metoder:
-   Normalapproximation – lämplig vid stora stickprov då medelvärdet närmar sig normalfördelning enligt centrala gränsvärdessatsen.
-   Bootstrap percentil – en icke-parametrisk metod som inte kräver normalitet och baseras på resampling av datan.
-   Bootstrap BCa – en förbättrad bootstrapvariant som korrigerar för bias och skevhet och kan ge mer tillförlitliga intervall vid snedfördelade data.

Samtliga metoder gav nästan identiska konfidensintervall, vilket visar att skattningen är robust. Normalapproximationen bedöms därför fungera väl i detta fall, samtidigt som bootstrapmetoderna ger en viktig kontroll utan parameterrisker.

Källor
-   Wasserman, L. (2004). All of Statistics. Springer.
-   Efron, B., & Tibshirani, R. J. (1993). An Introduction to the Bootstrap. Chapman & Hall/CRC.

### Hypotesprövning
För att testa om rökare har högre systoliskt blodtryck än icke-rökare användes två metoder:
1. Welch’s t-test
Welch-testet valdes eftersom grupperna har olika storlek och kan ha olika varians, vilket gör det mer robust än det klassiska t-testet vid heterogen spridning. Testet kräver inte antagandet om lika varians och anses vara standard vid jämförelse av två oberoende grupper med okända och olika variansparametrar (Welch, 1947).

2. Bootstrap-test av medelskillnaden
Som komplement användes ett icke-parametriskt bootstrap-test baserat på resampling av gruppernas data. Metoden kräver inte normalfördelning och ger en direkt empirisk uppskattning av p-värde och konfidensintervall (Efron & Tibshirani, 1993).

Kombinationen av Welch’s t-test och bootstrap ger en robust prövning av hypotesen och möjliggör kontroll av resultaten även om normalantagandet inte skulle hålla.

Källor
-   Welch, B. L. (1947). The generalization of “Student’s” problem…
-   Efron, B., & Tibshirani, R. J. (1993). An Introduction to the Bootstrap.

### Poweranalys
För att bedöma studiens statistiska styrka genomfördes både simulerad och analytisk poweranalys:
-   Simulerad power för det faktiska stickprovet:
Power uppskattades med resampling baserat på de observerade blodtrycksvärdena för rökare och icke-rökare. I varje simulering drogs nya urval inom respektive grupp, en effektskillnad Δ lades till gruppen rökare och ett Welch’s t-test (ensidigt, α = 0,05) beräknades. Andelen simuleringar med p < 0,05 användes som skattning av power för det faktiska N (= 800) och den observerade effekten (Δ ≈ 0,47 mmHg). Denna typ av resampling-baserad poweranalys rekommenderas när man vill bevara den verkliga datans spridning och obalans mellan grupper.
-   Simulerad power som funktion av effektstorlek och stickprovsstorlek:
Samma metod användes för att simulera power vid olika effektskillnader (Δ) med nuvarande N, samt vid olika totala stickprovsstorlekar med bibehållen gruppfördelning. Detta gav en bild av hur power förändras om effekten vore större eller studien vore större.

-   Analytiska beräkningar (normalapproximation):
Med antagande om normalfördelning användes standardformler för skillnad i två medelvärden för att beräkna ungefär
    (i) hur stor effekt som skulle krävas för 80 % power givet nuvarande N, samt
    (ii) hur stort N som skulle krävas för 80 % power givet den observerade effekten.

Dessa beräkningar fungerar som ett teoretiskt komplement till simuleringen.

Källor: 
-   Good, P. (2006). Resampling Methods: A Practical Guide to Data Analysis.
-   Wasserman, L. (2004). All of Statistics.
-   Efron, B., & Tibshirani, R. (1993). An Introduction to the Bootstrap.
